In [ ]:
import pandas as pd

In [ ]:
import glob
from zipfile import ZipFile

def init_dataframe_from_zip(path, index_col):
    df_master = pd.DataFrame()
    flag = False
    files_in_folder = glob.glob(path)
    for filename in files_in_folder:    
        zip_file = ZipFile(filename)
        for text_file in zip_file.infolist():
            if text_file.filename.endswith('.csv'):
                df = pd.read_csv(zip_file.open(text_file.filename),header=0,index_col=[index_col])
            if not flag:
                df_master = df
                flag = True
            else:
                df_master = pd.concat([df_master, df])
    return df_master

csse_covid_19_daily_reports_us = init_dataframe_from_zip(r'**csv_files/csse_covid_19_daily_reports_us.zip', 'UID')
display(csse_covid_19_daily_reports_us)
